In [1]:
import pytz
import os


DEFAULT_TZ = pytz.FixedOffset(540)  # GMT+09:00; Asia/Seoul

PATH_DATA = 'data/D'
PATH_ESM = os.path.join(PATH_DATA, 'EsmResponse.csv')
PATH_PARTICIPANT = os.path.join(PATH_DATA, 'UserInfo.csv')
PATH_SENSOR = os.path.join(PATH_DATA, 'Sensor')

PATH_INTERMEDIATE = os.path.join('/home/user/Collab/Data_Processing_D1/data/intermediate')
PATH_SAVE = '/home/user/Collab/MT/logo/3_clusters/data_MT'

DATA_TYPES = {
    'Acceleration': 'ACC',
    'AmbientLight': 'AML',
    'Calorie': 'CAL',
    'Distance': 'DST',
    'EDA': 'EDA',
    'HR': 'HRT',
    'RRI': 'RRI',
    'SkinTemperature': 'SKT',
    'StepCount': 'STP',
    'UltraViolet': 'ULV',
    'ActivityEvent': 'ACE',
    'ActivityTransition': 'ACT',
    'AppUsageEvent': 'APP',
    'BatteryEvent': 'BAT',
    'CallEvent': 'CAE',
    'Connectivity': 'CON',
    'DataTraffic': 'DAT',
    'InstalledApp': 'INS',
    'Location': 'LOC',
    'MediaEvent': 'MED',
    'MessageEvent': 'MSG',
    'WiFi': 'WIF',
    'ScreenEvent': 'SCR',
    'RingerModeEvent': 'RNG',
    'ChargeEvent': 'CHG',
    'PowerSaveEvent': 'PWS',
    'OnOffEvent': 'ONF'
}

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import cloudpickle
import ray
from datetime import datetime
from contextlib import contextmanager
import warnings
import time


def load(path: str):
    with open(path, mode='rb') as f:
        return cloudpickle.load(f)

    
def dump(obj, path: str):
    with open(path, mode='wb') as f:
        cloudpickle.dump(obj, f)
        
    
def log(msg: any):
    print('[{}] {}'.format(datetime.now().strftime('%y-%m-%d %H:%M:%S'), msg))


def summary(x):
    x = np.asarray(x)
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        n = len(x)
        # Here, uppercase np.dtype.kind corresponds to non-numeric data.
        # Also, we view the boolean data as dichotomous categorical data.
        if x.dtype.kind.isupper() or x.dtype.kind == 'b': 
            cnt = pd.Series(x).value_counts(dropna=False)
            card = len(cnt)
            cnt = cnt[:20]                
            cnt_str = ', '.join([f'{u}:{c}' for u, c in zip(cnt.index, cnt)])
            if card > 30:
                cnt_str = f'{cnt_str}, ...'
            return {
                'n': n,
                'cardinality': card,
                'value_count': cnt_str
            }
        else: 
            x_nan = x[np.isnan(x)]
            x_norm = x[~np.isnan(x)]
            
            tot = np.sum(x_norm)
            m = np.mean(x_norm)
            me = np.median(x_norm)
            s = np.std(x_norm, ddof=1)
            l, u = np.min(x_norm), np.max(x)
            conf_l, conf_u = st.t.interval(0.95, len(x_norm) - 1, loc=m, scale=st.sem(x_norm))
            n_nan = len(x_nan)
            
            return {
                'n': n,
                'sum': tot,
                'mean': m,
                'SD': s,
                'med': me,
                'range': (l, u),
                'conf.': (conf_l, conf_u),
                'nan_count': n_nan
            }


@contextmanager
def on_ray(*args, **kwargs):
    try:
        if ray.is_initialized():
            ray.shutdown()
        ray.init(*args, **kwargs)
        yield None
    finally:
        ray.shutdown()

transform = {
    'GAME': 'ENTER',
    'GAME_TRIVIA': 'ENTER',
    'GAME_CASINO': 'ENTER',
    'GAME-ACTION': 'ENTER',
    'GAME_SPORTS': 'ENTER',
    'GAME_PUZZLE': 'ENTER',
    'GAME_SIMULATION': 'ENTER',
    'GAME_STRATEGY': 'ENTER',
    'GAME_ROLE_PLAYING': 'ENTER',
    'GAME_ACTION': 'ENTER',
    'GAME_ARCADE': 'ENTER',
    'GAME_RACING': 'ENTER',
    'GAME_CASUAL': 'ENTER',
    'GAME_MUSIC': 'ENTER',
    'GAME_CARD': 'ENTER',
    'GAME_ADVENTURE': 'ENTER',
    'GAME_BOARD': 'ENTER',
    'GAME_EDUCATIONAL': 'ENTER',
    'GAME_RACING': 'ENTER',
    'PHOTOGRAPHY': 'ENTER',
    'ENTERTAINMENT': 'ENTER',
    'SPORTS': 'ENTER',
    'MUSIC_AND_AUDIO': 'ENTER',
    'COMICS': 'ENTER',
    'VIDEO_PLAYERS_AND_EDITORS': 'ENTER',
    'VIDEO_PLAYERS': 'ENTER',
    'ART_AND_DESIGN': 'ENTER',
    'TRAVEL_AND_LOCAL': 'INFO',
    'FOOD_AND_DRINK': 'INFO',
    'NEWS_AND_MAGAZINES': 'INFO',
    'MAPS_AND_NAVIGATION': 'INFO',
    'WEATHER': 'INFO',
    'HOUSE_AND_HOME': 'INFO',
    'BOOKS_AND_REFERENCE': 'INFO',
    'SHOPPING': 'INFO',
    'LIBRARIES_AND_DEMO': 'INFO',
    'BEAUTY': 'INFO',
    'AUTO_AND_VEHICLES': 'INFO',
    'LIFESTYLE': 'INFO',
    'PERSONALIZATION': 'SYSTEM',
    'TOOLS': 'SYSTEM',
    'COMMUNICATION': 'SOCIAL',
    'SOCIAL': 'SOCIAL',
    'DATING': 'SOCIAL',
    'PARENTING':'SOCIAL',
    'FINANCE': 'WORK',
    'BUSINESS': 'WORK',
    'PRODUCTIVITY': 'WORK',
    'EDUCATION': 'WORK',
    'HEALTH_AND_FITNESS': 'HEALTH',
    'MEDICAL': 'HEALTH',
    'SYSTEM': 'SYSTEM',
    'MISC': 'SYSTEM', # ABC logger
     None: 'UNKNOWN',
    'UNKNOWN':'UNKNOWN'
}

In [3]:
p = os.path.join(PATH_INTERMEDIATE, 'stress-fixed.pkl')
X, y, groups, t, datetimes = load(p)

In [4]:
PARTICIPANTS = pd.read_csv(os.path.join(PATH_INTERMEDIATE, 'PARTICIPANT_INFO.csv'),index_col = 'pcode')
PINFO = PARTICIPANTS.assign(
    BFI_OPN=lambda x: x['openness'],
    BFI_CON=lambda x: x['conscientiousness'],
    BFI_NEU=lambda x: x['neuroticism'],
    BFI_EXT=lambda x: x['extraversion'],
    BFI_AGR=lambda x: x['agreeableness'],
)[[
    'BFI_OPN', 'BFI_CON', 'BFI_NEU', 'BFI_EXT', 'BFI_AGR'
]]
PINFO = pd.get_dummies(PINFO, prefix_sep='=', dtype=bool)

In [5]:
LABELS_PROC = pd.read_csv(os.path.join(PATH_INTERMEDIATE, 'LABELS_PROC.csv'), index_col=['pcode','timestamp'],parse_dates=True)

In [6]:
_df =LABELS_PROC
_df.reset_index(level='timestamp', inplace=True)
print('First timestamp:', _df['timestamp'].min())
print('Last timestamp:', _df['timestamp'].max())

First timestamp: 2019-04-30 10:03:28+09:00
Last timestamp: 2019-05-22 22:02:03+09:00


In [7]:
time_ranges = _df.groupby('pcode')['timestamp'].agg(['min', 'max'])

In [8]:
list_pid = set(LABELS_PROC.index.get_level_values('pcode').values)

In [9]:
PINFO_valid = PINFO.loc[PINFO.index.isin(list_pid)]

In [10]:
#Divide the features into different categories
feat_current = X.loc[:,[('#VAL' in str(x)) or ('ESM#LastLabel' in str(x)) for x in X.keys()]]  
feat_dsc = X.loc[:,[('#DSC' in str(x))  for x in X.keys()]]  
feat_yesterday = X.loc[:,[('Yesterday' in str(x))  for x in X.keys()]]  
feat_today = X.loc[:,[('Today' in str(x))  for x in X.keys()]]  
feat_sleep = X.loc[:,[('Sleep' in str(x))  for x in X.keys()]]  
feat_time = X.loc[:,[('Time' in str(x))  for x in X.keys()]]  
feat_pif = X.loc[:,[('PIF' in str(x))  for x in X.keys()]]  
feat_ImmediatePast = X.loc[:,[('ImmediatePast_15' in str(x))  for x in X.keys()]]
#Divide the time window features into sensor/past stress label
feat_current_sensor = X.loc[:,[('#VAL' in str(x))  for x in X.keys()]]  
feat_current_ESM = X.loc[:,[('ESM#LastLabel' in str(x)) for x in X.keys()]]  
feat_ImmediatePast_sensor = feat_ImmediatePast.loc[:,[('ESM' not in str(x)) for x in feat_ImmediatePast.keys()]]  
feat_ImmediatePast_ESM = feat_ImmediatePast.loc[:,[('ESM'  in str(x)) for x in feat_ImmediatePast.keys()]]  
feat_today_sensor = feat_today.loc[:,[('ESM' not in str(x))  for x in feat_today.keys()]]  
feat_today_ESM = feat_today.loc[:,[('ESM'  in str(x)) for x in feat_today.keys()]]  
feat_yesterday_sensor = feat_yesterday.loc[:,[('ESM' not in str(x)) for x in feat_yesterday.keys()]]  
feat_yesterday_ESM = feat_yesterday.loc[:,[('ESM'  in str(x)) for x in feat_yesterday.keys()]]
#Prepare the final feature set
feat_baseline = pd.concat([ feat_time,feat_dsc,feat_current_sensor, feat_ImmediatePast_sensor],axis=1)
feat_final = pd.concat([feat_baseline ], axis=1)
X = feat_final

In [11]:
similar_user = pd.read_csv(os.path.join(PATH_INTERMEDIATE,  'similar_user_5.csv'))

In [12]:
import pandas as pd

# Convert groups to a pandas DataFrame
groups_df = pd.DataFrame(groups, columns=['pcode'])

# Ensure the data types of the 'pcode' and 'pcode' columns are the same
groups_df['pcode'] = groups_df['pcode'].astype(str)
similar_user['pcode'] = similar_user['pcode'].astype(str)

# Create a new Series that maps Pcode to cluster label
cluster_map = similar_user.set_index('pcode')['cluster']

# Create the 'cluster' column in the 'groups' DataFrame
groups_df['cluster'] = groups_df['pcode'].map(cluster_map)

In [13]:
cluster_map.head()

pcode
P01    2
P02    3
P03    4
P05    3
P06    1
Name: cluster, dtype: int64

In [14]:
# Create a dictionary to store data structures for each cluster
clusters = {}

# Loop through each unique cluster label in the Series
for cluster_label in cluster_map.unique():
    clusters[cluster_label] = cluster_map[cluster_map == cluster_label]

In [15]:
clusters[4]

pcode
P03    4
P12    4
P13    4
P15    4
P19    4
P32    4
P40    4
P50    4
P57    4
P61    4
P75    4
Name: cluster, dtype: int64

In [16]:
# XGBoost parameters
param = {
    "early_stopping_rounds": 200,
    "reg_alpha": 0,
    "colsample_bytree": 1,
    "colsample_bylevel": 1,
    "scale_pos_weight": 1,
    "learning_rate": 0.3,
    "nthread": 10,
    "min_child_weight": 1,
    "n_estimators": 1000,
    "subsample": 1,
    "reg_lambda": 1,
    "seed": 27,
    "objective": 'binary:logistic',
    "max_depth": 6,
    "gamma": 0,
    'eval_metric': 'auc',
    'silent': 1,
    'tree_method': 'exact',
    'debug': 0,
    'use_task_gain_self': 0,
    'when_task_split': 1,
    'how_task_split': 0,
    'min_task_gain': 0.0,
    'task_gain_margin': 0.0,
    'max_neg_sample_ratio': 0.4,
    'which_task_value': 2,
    'baseline_alpha': 1.0,
    'baseline_lambda': 1.0,
}

In [21]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, precision_score

# Store the results
results = {}

# Iterate through each cluster
for cluster_label, cluster_data in clusters.items():
    results[cluster_label] = {'accuracy': [], 'auc': [], 'f1': [], 'recall': [], 'F1 Positive': [], 'Precision': []}
    
    # Users in current cluster
    users_in_cluster = cluster_data.index.tolist()
    
    # Create a mask for data instances belonging to this cluster
    cluster_data_mask = groups_df['pcode'].isin(users_in_cluster)
    
    # Iterate through each user in the cluster for LOUO cross-validation
    for user in users_in_cluster:
        
        # Find the indices in groups_df for this user
        user_indices = groups_df[groups_df['pcode'] == user].index.tolist()
        
        # Mask for excluding the current user's data
        train_mask = (cluster_data_mask) & (~groups_df.index.isin(user_indices))
        
        # Split the data for LOUO cross-validation
        X_train = X[train_mask]
        y_train = y[train_mask.values]
        
        X_validate = X.loc[user_indices]
        y_validate = y[user_indices]
        
        # Train the XGBoost model
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalidate = xgb.DMatrix(X_validate, label=y_validate)
        
        evallist = [(dtrain, 'train'), (dvalidate, 'test')]
        bst = xgb.train(param, dtrain, num_boost_round=1000, early_stopping_rounds=200, evals=evallist)
        
        # Predict on validation set
        preds_prob = bst.predict(dvalidate, ntree_limit=bst.best_ntree_limit)
        preds = [1 if p > 0.5 else 0 for p in preds_prob]
        
        # Calculate the metrics
        accuracy = accuracy_score(y_validate, preds)
        auc = roc_auc_score(y_validate, preds_prob)
        f1 = f1_score(y_validate, preds, average='macro')
        recall = recall_score(y_validate, preds)
        f1p = f1_score(y_validate, preds, pos_label=1)
        precision = precision_score(y_validate, preds)
        
        # Store the metrics
        results[cluster_label]['accuracy'].append(accuracy)
        results[cluster_label]['auc'].append(auc)
        results[cluster_label]['f1'].append(f1)
        results[cluster_label]['recall'].append(recall)
        results[cluster_label]['F1 Positive'].append(f1p)
        results[cluster_label]['Precision'].append(precision)

# results dictionary will have the metrics for each user in each cluster

[13:45:07] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:767: 
Parameters: { "baseline_alpha", "baseline_lambda", "debug", "early_stopping_rounds", "how_task_split", "max_neg_sample_ratio", "min_task_gain", "n_estimators", "silent", "task_gain_margin", "use_task_gain_self", "when_task_split", "which_task_value" } are not used.

[0]	train-auc:0.93033	test-auc:0.43103
[1]	train-auc:0.96988	test-auc:0.43417
[2]	train-auc:0.99295	test-auc:0.47492
[3]	train-auc:0.99230	test-auc:0.45925
[4]	train-auc:0.99830	test-auc:0.48119
[5]	train-auc:0.99909	test-auc:0.43887
[6]	train-auc:0.99967	test-auc:0.47806
[7]	train-auc:1.00000	test-auc:0.51724
[8]	train-auc:1.00000	test-auc:0.52978
[9]	train-auc:1.00000	test-auc:0.52978
[10]	train-auc:1.00000	test-auc:0.52351
[11]	train-auc:1.00000	test-auc:0.53605
[12]	train-auc:1.00000	test-auc:0.52351
[13]	train-auc:1.00000	test-auc:0.52665
[14]	train-auc:1.00000	test-auc:0.53292
[15]	train-auc:1.00000	tes

[194]	train-auc:1.00000	test-auc:0.50470
[195]	train-auc:1.00000	test-auc:0.50470
[196]	train-auc:1.00000	test-auc:0.50470
[197]	train-auc:1.00000	test-auc:0.50470
[198]	train-auc:1.00000	test-auc:0.50470
[199]	train-auc:1.00000	test-auc:0.50470
[200]	train-auc:1.00000	test-auc:0.50470
[201]	train-auc:1.00000	test-auc:0.50157
[202]	train-auc:1.00000	test-auc:0.50157
[203]	train-auc:1.00000	test-auc:0.50157
[204]	train-auc:1.00000	test-auc:0.50157
[205]	train-auc:1.00000	test-auc:0.50157
[206]	train-auc:1.00000	test-auc:0.50157
[207]	train-auc:1.00000	test-auc:0.50157
[208]	train-auc:1.00000	test-auc:0.50157
[209]	train-auc:1.00000	test-auc:0.50157
[210]	train-auc:1.00000	test-auc:0.50157
[211]	train-auc:1.00000	test-auc:0.50157
[212]	train-auc:1.00000	test-auc:0.50157
[213]	train-auc:1.00000	test-auc:0.50157
[214]	train-auc:1.00000	test-auc:0.50157
[215]	train-auc:1.00000	test-auc:0.50470


/home/user/miniconda3/envs/sci-data/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


[13:45:17] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:767: 
Parameters: { "baseline_alpha", "baseline_lambda", "debug", "early_stopping_rounds", "how_task_split", "max_neg_sample_ratio", "min_task_gain", "n_estimators", "silent", "task_gain_margin", "use_task_gain_self", "when_task_split", "which_task_value" } are not used.

[0]	train-auc:0.94412	test-auc:0.56944
[1]	train-auc:0.97785	test-auc:0.70556
[2]	train-auc:0.99421	test-auc:0.68611
[3]	train-auc:0.99935	test-auc:0.67222
[4]	train-auc:0.99961	test-auc:0.61667
[5]	train-auc:1.00000	test-auc:0.64444
[6]	train-auc:1.00000	test-auc:0.57222
[7]	train-auc:1.00000	test-auc:0.52222
[8]	train-auc:1.00000	test-auc:0.55556
[9]	train-auc:1.00000	test-auc:0.54444
[10]	train-auc:1.00000	test-auc:0.55556
[11]	train-auc:1.00000	test-auc:0.58333
[12]	train-auc:1.00000	test-auc:0.58889
[13]	train-auc:1.00000	test-auc:0.57222
[14]	train-auc:1.00000	test-auc:0.55000
[15]	train-auc:1.00000	tes

[194]	train-auc:1.00000	test-auc:0.42778
[195]	train-auc:1.00000	test-auc:0.42222
[196]	train-auc:1.00000	test-auc:0.42222
[197]	train-auc:1.00000	test-auc:0.42778
[198]	train-auc:1.00000	test-auc:0.42778
[199]	train-auc:1.00000	test-auc:0.42778
[200]	train-auc:1.00000	test-auc:0.42778
[201]	train-auc:1.00000	test-auc:0.43333


...

[194]	train-auc:1.00000	test-auc:0.50667
[195]	train-auc:1.00000	test-auc:0.50667
[196]	train-auc:1.00000	test-auc:0.50667
[197]	train-auc:1.00000	test-auc:0.50667
[198]	train-auc:1.00000	test-auc:0.50519
[199]	train-auc:1.00000	test-auc:0.50667
[200]	train-auc:1.00000	test-auc:0.50815
[201]	train-auc:1.00000	test-auc:0.50963


/home/user/miniconda3/envs/sci-data/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


[13:54:51] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:767: 
Parameters: { "baseline_alpha", "baseline_lambda", "debug", "early_stopping_rounds", "how_task_split", "max_neg_sample_ratio", "min_task_gain", "n_estimators", "silent", "task_gain_margin", "use_task_gain_self", "when_task_split", "which_task_value" } are not used.

[0]	train-auc:0.87592	test-auc:0.40848
[1]	train-auc:0.94492	test-auc:0.37723
[2]	train-auc:0.96890	test-auc:0.43527
[3]	train-auc:0.98845	test-auc:0.42857
[4]	train-auc:0.99673	test-auc:0.41518
[5]	train-auc:0.99798	test-auc:0.44643
[6]	train-auc:0.99947	test-auc:0.42857
[7]	train-auc:0.99984	test-auc:0.41071
[8]	train-auc:0.99998	test-auc:0.40179
[9]	train-auc:0.99999	test-auc:0.43304
[10]	train-auc:1.00000	test-auc:0.41518
[11]	train-auc:1.00000	test-auc:0.43750
[12]	train-auc:1.00000	test-auc:0.48214
[13]	train-auc:1.00000	test-auc:0.49554
[14]	train-auc:1.00000	test-auc:0.51339
[15]	train-auc:1.00000	tes

[194]	train-auc:1.00000	test-auc:0.45536
[195]	train-auc:1.00000	test-auc:0.45982
[196]	train-auc:1.00000	test-auc:0.45982
[197]	train-auc:1.00000	test-auc:0.45982
[198]	train-auc:1.00000	test-auc:0.46429
[199]	train-auc:1.00000	test-auc:0.46429
[200]	train-auc:1.00000	test-auc:0.46429
[201]	train-auc:1.00000	test-auc:0.46429
[202]	train-auc:1.00000	test-auc:0.45982
[203]	train-auc:1.00000	test-auc:0.45089
[204]	train-auc:1.00000	test-auc:0.44643
[205]	train-auc:1.00000	test-auc:0.44643
[206]	train-auc:1.00000	test-auc:0.44643
[207]	train-auc:1.00000	test-auc:0.45089
[208]	train-auc:1.00000	test-auc:0.45089
[209]	train-auc:1.00000	test-auc:0.45089
[210]	train-auc:1.00000	test-auc:0.45089
[211]	train-auc:1.00000	test-auc:0.45089
[212]	train-auc:1.00000	test-auc:0.45536
[213]	train-auc:1.00000	test-auc:0.45536
[214]	train-auc:1.00000	test-auc:0.45089


/home/user/miniconda3/envs/sci-data/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [24]:
# Store average results for each cluster
average_results_per_cluster = {}

for cluster_label, metrics in results.items():
    average_results_per_cluster[cluster_label] = {
        'avg_accuracy': sum(metrics['accuracy']) / len(metrics['accuracy']),
        'avg_auc': sum(metrics['auc']) / len(metrics['auc']),
        'avg_f1': sum(metrics['f1']) / len(metrics['f1']),
        'avg_recall': sum(metrics['recall']) / len(metrics['recall']),
        'avg_fp1':sum(metrics['F1 Positive']) / len(metrics['F1 Positive']),
        'avg_pre':sum(metrics['Precision']) / len(metrics['Precision'])
    }

# Calculate overall averages
all_accuracies = [avg['avg_accuracy'] for cluster, avg in average_results_per_cluster.items()]
all_aucs = [avg['avg_auc'] for cluster, avg in average_results_per_cluster.items()]
all_f1s = [avg['avg_f1'] for cluster, avg in average_results_per_cluster.items()]
all_recalls = [avg['avg_recall'] for cluster, avg in average_results_per_cluster.items()]
all_fp1 = [avg['avg_fp1'] for cluster, avg in average_results_per_cluster.items()]
all_pre = [avg['avg_pre'] for cluster, avg in average_results_per_cluster.items()]

overall_avg_results = {
    'overall_avg_accuracy': sum(all_accuracies) / len(all_accuracies),
    'overall_avg_auc': sum(all_aucs) / len(all_aucs),
    'overall_avg_f1': sum(all_f1s) / len(all_f1s),
    'overall_avg_recall': sum(all_recalls) / len(all_recalls),
    'overall_avg_f1p' : sum(all_fp1) / len(all_fp1),
    'overall_avg_pre' : sum(all_pre) / len(all_pre)
}

print(average_results_per_cluster)
print(overall_avg_results)

{2: {'avg_accuracy': 0.6446662057299415, 'avg_auc': 0.6130530734697403, 'avg_f1': 0.4753775920025263, 'avg_recall': 0.21413031413031414, 'avg_fp1': 0.18299965600275195, 'avg_pre': 0.26782051282051283}, 3: {'avg_accuracy': 0.6008613140293421, 'avg_auc': 0.6297346320871432, 'avg_f1': 0.5211212086124876, 'avg_recall': 0.3380967299200382, 'avg_fp1': 0.33774382420813437, 'avg_pre': 0.4670506956098089}, 4: {'avg_accuracy': 0.7094176026092903, 'avg_auc': 0.6693392215604502, 'avg_f1': 0.4870095477464881, 'avg_recall': 0.14529914529914528, 'avg_fp1': 0.15509348336672815, 'avg_pre': 0.21028708133971288}, 1: {'avg_accuracy': 0.5957714907825479, 'avg_auc': 0.5122062190107304, 'avg_f1': 0.4315929113511564, 'avg_recall': 0.09947089947089946, 'avg_fp1': 0.129756770743488, 'avg_pre': 0.2162087912087912}, 0: {'avg_accuracy': 0.4882618404286743, 'avg_auc': 0.5503419824676633, 'avg_f1': 0.4264147367985403, 'avg_recall': 0.3794420252145775, 'avg_fp1': 0.31377752086316046, 'avg_pre': 0.47063387734743234}}
